In [1]:
import numpy as np 
import cv2

In [2]:
def detect_and_blur(img, plate_rects):
    
    # Watch solutions video for line by line explanation!
    
    plate_img = img.copy()
    roi = img.copy()
    
    for (x,y,w,h) in plate_rects: 
        
        roi = roi[y:y+h,x:x+w]
        blurred_roi = cv2.medianBlur(roi,7)
        
        plate_img[y:y+h,x:x+w] = blurred_roi
        
    return plate_img

In [3]:
#Capturing the video
cap = cv2.VideoCapture(0)
#Reading the video frame by frame
ret, frame = cap.read()
#Using frontal face cascade from opencv
face_cascade = cv2.CascadeClassifier("../DATA/haarcascades/haarcascade_frontalface_default.xml")

face_rects = face_cascade.detectMultiScale(frame)
#Getting the lengths for the rectangle
(face_x, face_y, w, h) = tuple(face_rects[0])
track_window = (face_x, face_y, w, h)
#Getting the region of interest using the lengths that are obtained
roi = frame[face_y:face_y+h, face_x:face_x+w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
#Calculating histogram of set of arrays
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])
#Normalizing between 0 and 255
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()
    
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1)
        
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        # Getting the frame and drawing a rectangle around the face and later blurring it
        x,y,w,h =track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 5)
        img2 = detect_and_blur(img2, face_rects)
        #Seeing the image
        cv2.imshow("img", img2)
        # Press Esc to leave
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    else:
        break
            
cap.release()
cv2.destroyAllWindows()
